# Imports

In [1]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import sklearn
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier

# Train

In [2]:
train_set = pd.read_csv('train/train_encoded.csv')

In [3]:
train_set

,len_location,len_keyword,len_text,len_text_original,diff_len_text,keyword_value,keyword_sum,keyword_min,keyword_max,keyword_mean,...,text_mean_tf-idf,text_median_tf-idf,text_value,text_norm_value,text_sum_w2v,text_min_w2v,text_max_w2v,text_mean_w2v,text_median_w2v,target
0,0,0,43,69,26,3.267254,1.0,1.0,1.0,1.0,...,0.343243,0.323476,27.431596,12.513938,8.481658,0.000850,0.615265,0.154212,0.121727,1
1,0,0,36,38,2,3.267254,1.0,1.0,1.0,1.0,...,0.363618,0.340166,15.093579,7.808587,0.790154,-0.103024,0.308946,0.079015,0.055061,1
2,0,0,72,133,61,3.267254,1.0,1.0,1.0,1.0,...,0.327084,0.256037,51.178261,21.319380,29.265612,-0.044882,1.000000,0.139360,0.095317,1
3,0,0,50,65,15,3.267254,1.0,1.0,1.0,1.0,...,0.359336,0.346962,24.392019,11.505696,2.733583,-0.039545,0.317247,0.097628,0.082558,1
4,0,0,52,88,36,3.267254,1.0,1.0,1.0,1.0,...,0.326443,0.287533,38.645027,15.708514,13.153737,-0.041156,1.000000,0.125274,0.086994,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0,0,50,83,33,3.267254,1.0,1.0,1.0,1.0,...,0.320506,0.352132,23.781937,10.863061,3.762588,-0.026625,0.312744,0.104516,0.090719,1
7609,0,0,82,125,43,3.267254,1.0,1.0,1.0,1.0,...,0.293544,0.279435,33.849820,15.519987,25.374036,-0.006511,1.000000,0.211450,0.158652,1
7610,0,0,60,65,5,3.267254,1.0,1.0,1.0,1.0,...,0.477137,0.527716,30.685293,15.395339,13.296097,-0.082743,1.000000,0.201456,0.046951,1
7611,0,0,96,137,41,3.267254,1.0,1.0,1.0,1.0,...,0.265336,0.228318,52.121597,21.445930,22.747623,-0.091922,1.000000,0.119724,0.085732,1


In [4]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]

## Bagging

In [5]:
lgbm_model = lgb.LGBMClassifier()

model = BaggingClassifier(base_estimator=lgbm_model)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)
results = model_selection.cross_val_score(model, X, y, cv=cv, scoring='f1')

print(results.mean())

0.6695220851340706


In [6]:
lgbm_model = lgb.LGBMClassifier(objective='binary', num_leaves=40, learning_rate=0.1, max_depth=100)

model = BaggingClassifier(base_estimator=lgbm_model)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)
results = model_selection.cross_val_score(model, X, y, cv=cv, scoring='f1')

print(results.mean())

0.6707439035196407


In [7]:
lgbm_model = lgb.LGBMClassifier(objective='binary', num_leaves=40, learning_rate=0.1, max_depth=100)

model = BaggingClassifier(base_estimator=lgbm_model)

cv = RepeatedStratifiedKFold(random_state=42)
results = model_selection.cross_val_score(model, X, y, cv=cv, scoring='f1')

print(results.mean())

0.6685482053852124


In [8]:
seed = 7
num_trees = 100
kfold = model_selection.KFold(n_splits=2)
model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)

results = model_selection.cross_val_score(model, X, y, cv=kfold)
print(results.mean())

0.6519122682571212


In [9]:
kfold = model_selection.KFold(n_splits=3)
estimators = []

model1 = lgb.LGBMClassifier(objective='binary', num_leaves=20, learning_rate=0.1)
estimators.append(('lgbm', model1))

model2 = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.01, max_depth = 100, n_estimators = 100)
estimators.append(('xgb', model2))

model3 = CatBoostClassifier(silent=True)
estimators.append(('catboost', model3))

model4 = AdaBoostClassifier(n_estimators=100, random_state=42)
estimators.append(('adaboost', model4))

ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, X, y, cv=kfold)
print(results.mean())

0.6767393922715028


0.6081618329246296

In [10]:
kfold = model_selection.KFold(n_splits=2)
estimators = []

model1 = lgb.LGBMClassifier()
estimators.append(('lgbm', model1))

model2 = xgb.XGBClassifier()
estimators.append(('xgb', model2))

model3 = CatBoostClassifier(silent=True)
estimators.append(('catboost', model3))

ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, X, y, cv=kfold)
print(results.mean())

0.6616318972117767


In [11]:
kfold = model_selection.KFold(n_splits=10)
estimators = []


model4 = AdaBoostClassifier(n_estimators=100, random_state=42)
estimators.append(('adaboost', model4))

model1 = lgb.LGBMClassifier(objective='binary', num_leaves=20, learning_rate=0.1)
estimators.append(('lgbm', model1))

model3 = CatBoostClassifier(silent=True)
estimators.append(('catboost', model3))

model2 = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.01, max_depth = 100, n_estimators = 100)
estimators.append(('xgb', model2))

ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, X, y, cv=kfold)
print(results.mean())

0.6750238841695378


In [12]:
results

array([0.67716535, 0.6824147 , 0.70866142, 0.63206307, 0.61498029,
       0.64520368, 0.70302234, 0.66228647, 0.68068331, 0.74375821])